In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import scipy.io
from sklearn.preprocessing import OneHotEncoder

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})

In [12]:
features = x['filled_features']
print("Features: ", features.shape, type(features))
print(features)

Features:  (45810, 113) <class 'numpy.ndarray'>
[[ -43.  971. -339. ...   29.  -24.  165.]
 [ -33.  957. -347. ...    5.  -33.  165.]
 [ -35.  966. -363. ...   32.   47.  165.]
 ...
 [-313.  905.  656. ...   15.  -44.  -76.]
 [-232.  959.  695. ...   19.  -10.  -76.]
 [  -9.  950.  791. ...    2.   70.  -76.]]


In [11]:
labels = x['labels'][:,0]
print("Labels: ", labels.shape, type(labels), "\n", labels)
onehot_encoder = OneHotEncoder(sparse=False)
labels = onehot_encoder.fit_transform(x['labels'][:,0].reshape(-1, 1))
print("\nOne hot encoded labels: ", labels.shape, type(labels), "\n", labels)

Labels:  (45810,) <class 'numpy.ndarray'> 
 [1 1 1 ... 2 2 2]

One hot encoded labels:  (45810, 5) <class 'numpy.ndarray'> 
 [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


## Dataset definition ##
Here we take time windows of 50 samples, with a sliding step (stride) of 25 samples, and each sample is stored as an element of the Dataset.

In [28]:
window_size = 50
stride = window_size / 2
num_features = features.shape[1]
samples, classes = labels.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

Number of samples:  45810 
Number of classes:  5


In [37]:
windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of windows:  1831


Dataset should have size 50 x 113 x 1832

In [65]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(1831, 50, 113)

In [66]:
lab_cum = np.zeros([windows])
lab_cum.shape

(1831,)

In [67]:
for w in range(windows):
    index = int(w * stride)
    print(w, index)
    cube[w,:,:] = features[index:index+window_size, :]
    l = labels[index:index+window_size,:] # shape 50 x 5
    lab_cum[w] = np.argmax(np.sum(l, axis=0))
    print(np.sum(l, axis=0), lab_cum[w])

0 0
[ 0. 50.  0.  0.  0.] 1.0
1 25
[ 0. 50.  0.  0.  0.] 1.0
2 50
[ 0. 50.  0.  0.  0.] 1.0
3 75
[ 0. 50.  0.  0.  0.] 1.0
4 100
[ 0. 50.  0.  0.  0.] 1.0
5 125
[ 0. 50.  0.  0.  0.] 1.0
6 150
[ 0. 50.  0.  0.  0.] 1.0
7 175
[ 0. 50.  0.  0.  0.] 1.0
8 200
[ 0. 50.  0.  0.  0.] 1.0
9 225
[ 0. 50.  0.  0.  0.] 1.0
10 250
[ 0. 50.  0.  0.  0.] 1.0
11 275
[ 0. 50.  0.  0.  0.] 1.0
12 300
[ 0. 50.  0.  0.  0.] 1.0
13 325
[ 0. 50.  0.  0.  0.] 1.0
14 350
[ 0. 50.  0.  0.  0.] 1.0
15 375
[ 0. 50.  0.  0.  0.] 1.0
16 400
[ 0. 50.  0.  0.  0.] 1.0
17 425
[ 0. 50.  0.  0.  0.] 1.0
18 450
[ 0. 50.  0.  0.  0.] 1.0
19 475
[ 0. 50.  0.  0.  0.] 1.0
20 500
[ 0. 50.  0.  0.  0.] 1.0
21 525
[ 0. 50.  0.  0.  0.] 1.0
22 550
[ 0. 50.  0.  0.  0.] 1.0
23 575
[ 0. 50.  0.  0.  0.] 1.0
24 600
[ 0. 50.  0.  0.  0.] 1.0
25 625
[ 0. 50.  0.  0.  0.] 1.0
26 650
[ 0. 50.  0.  0.  0.] 1.0
27 675
[ 0. 50.  0.  0.  0.] 1.0
28 700
[ 0. 50.  0.  0.  0.] 1.0
29 725
[ 0. 50.  0.  0.  0.] 1.0
30 750
[ 0. 50.  0.  0.  

[ 1. 47.  2.  0.  0.] 1.0
400 10000
[ 0. 50.  0.  0.  0.] 1.0
401 10025
[ 0. 50.  0.  0.  0.] 1.0
402 10050
[ 0. 50.  0.  0.  0.] 1.0
403 10075
[ 0. 50.  0.  0.  0.] 1.0
404 10100
[ 0. 50.  0.  0.  0.] 1.0
405 10125
[ 0. 50.  0.  0.  0.] 1.0
406 10150
[ 0. 50.  0.  0.  0.] 1.0
407 10175
[ 0. 50.  0.  0.  0.] 1.0
408 10200
[ 0. 50.  0.  0.  0.] 1.0
409 10225
[ 0. 50.  0.  0.  0.] 1.0
410 10250
[ 0. 50.  0.  0.  0.] 1.0
411 10275
[ 0. 50.  0.  0.  0.] 1.0
412 10300
[ 0. 50.  0.  0.  0.] 1.0
413 10325
[ 0. 50.  0.  0.  0.] 1.0
414 10350
[ 0. 50.  0.  0.  0.] 1.0
415 10375
[ 0. 50.  0.  0.  0.] 1.0
416 10400
[ 0. 50.  0.  0.  0.] 1.0
417 10425
[ 0. 50.  0.  0.  0.] 1.0
418 10450
[ 0. 50.  0.  0.  0.] 1.0
419 10475
[ 0. 50.  0.  0.  0.] 1.0
420 10500
[ 0. 50.  0.  0.  0.] 1.0
421 10525
[ 0. 25. 25.  0.  0.] 1.0
422 10550
[ 0.  0. 50.  0.  0.] 2.0
423 10575
[ 0.  0. 50.  0.  0.] 2.0
424 10600
[ 0.  0. 50.  0.  0.] 2.0
425 10625
[ 0.  0. 50.  0.  0.] 2.0
426 10650
[ 0.  0. 50.  0.  0.] 2.0
42

[ 0.  0. 50.  0.  0.] 2.0
775 19375
[ 0.  0. 50.  0.  0.] 2.0
776 19400
[ 0.  3. 47.  0.  0.] 2.0
777 19425
[ 0. 28. 22.  0.  0.] 1.0
778 19450
[ 0. 50.  0.  0.  0.] 1.0
779 19475
[ 0. 50.  0.  0.  0.] 1.0
780 19500
[ 0. 50.  0.  0.  0.] 1.0
781 19525
[ 0. 50.  0.  0.  0.] 1.0
782 19550
[12. 26. 12.  0.  0.] 1.0
783 19575
[15.  1. 34.  0.  0.] 2.0
784 19600
[ 6. 22. 22.  0.  0.] 1.0
785 19625
[ 3. 47.  0.  0.  0.] 1.0
786 19650
[ 0. 50.  0.  0.  0.] 1.0
787 19675
[ 0. 50.  0.  0.  0.] 1.0
788 19700
[ 0. 50.  0.  0.  0.] 1.0
789 19725
[ 0. 50.  0.  0.  0.] 1.0
790 19750
[ 0. 50.  0.  0.  0.] 1.0
791 19775
[ 3. 31. 16.  0.  0.] 1.0
792 19800
[ 3.  6. 41.  0.  0.] 2.0
793 19825
[ 0.  0. 50.  0.  0.] 2.0
794 19850
[ 0.  0. 50.  0.  0.] 2.0
795 19875
[ 7.  6. 37.  0.  0.] 2.0
796 19900
[ 7. 31. 12.  0.  0.] 1.0
797 19925
[ 0. 50.  0.  0.  0.] 1.0
798 19950
[ 0. 50.  0.  0.  0.] 1.0
799 19975
[ 0. 50.  0.  0.  0.] 1.0
800 20000
[ 0. 27. 23.  0.  0.] 1.0
801 20025
[ 0.  2. 48.  0.  0.] 2.0
80

[ 0. 50.  0.  0.  0.] 1.0
1150 28750
[ 1. 48.  1.  0.  0.] 1.0
1151 28775
[ 1. 23. 26.  0.  0.] 2.0
1152 28800
[ 0.  0. 50.  0.  0.] 2.0
1153 28825
[ 0.  0. 50.  0.  0.] 2.0
1154 28850
[ 5. 20. 25.  0.  0.] 2.0
1155 28875
[ 5. 45.  0.  0.  0.] 1.0
1156 28900
[ 0. 50.  0.  0.  0.] 1.0
1157 28925
[ 0. 50.  0.  0.  0.] 1.0
1158 28950
[ 0. 50.  0.  0.  0.] 1.0
1159 28975
[ 0. 50.  0.  0.  0.] 1.0
1160 29000
[ 0. 50.  0.  0.  0.] 1.0
1161 29025
[ 0. 50.  0.  0.  0.] 1.0
1162 29050
[ 0. 50.  0.  0.  0.] 1.0
1163 29075
[ 0. 50.  0.  0.  0.] 1.0
1164 29100
[ 0. 50.  0.  0.  0.] 1.0
1165 29125
[ 0. 50.  0.  0.  0.] 1.0
1166 29150
[ 0. 50.  0.  0.  0.] 1.0
1167 29175
[ 0. 50.  0.  0.  0.] 1.0
1168 29200
[ 0. 50.  0.  0.  0.] 1.0
1169 29225
[ 0. 50.  0.  0.  0.] 1.0
1170 29250
[ 0. 50.  0.  0.  0.] 1.0
1171 29275
[ 0. 50.  0.  0.  0.] 1.0
1172 29300
[ 0. 50.  0.  0.  0.] 1.0
1173 29325
[ 0. 50.  0.  0.  0.] 1.0
1174 29350
[ 0. 50.  0.  0.  0.] 1.0
1175 29375
[ 0. 50.  0.  0.  0.] 1.0
1176 29400
[

1524 38100
[ 0.  0.  0. 50.  0.] 3.0
1525 38125
[ 0.  0.  0. 50.  0.] 3.0
1526 38150
[ 0.  0.  0. 50.  0.] 3.0
1527 38175
[ 0.  0.  0. 50.  0.] 3.0
1528 38200
[ 0.  0.  0. 50.  0.] 3.0
1529 38225
[ 0.  0.  0. 50.  0.] 3.0
1530 38250
[ 0.  0.  0. 50.  0.] 3.0
1531 38275
[ 0.  0.  0. 50.  0.] 3.0
1532 38300
[ 0.  0.  0. 50.  0.] 3.0
1533 38325
[ 0.  0.  0. 50.  0.] 3.0
1534 38350
[ 0.  0.  0. 50.  0.] 3.0
1535 38375
[ 0.  0.  0. 50.  0.] 3.0
1536 38400
[ 0.  0.  0. 50.  0.] 3.0
1537 38425
[ 0.  0.  0. 50.  0.] 3.0
1538 38450
[ 0.  0.  0. 50.  0.] 3.0
1539 38475
[ 0.  0.  0. 50.  0.] 3.0
1540 38500
[ 0.  0.  0. 50.  0.] 3.0
1541 38525
[ 0.  0.  0. 50.  0.] 3.0
1542 38550
[ 0.  0.  0. 50.  0.] 3.0
1543 38575
[ 0.  0.  0. 50.  0.] 3.0
1544 38600
[ 0.  0.  0. 50.  0.] 3.0
1545 38625
[ 0.  0.  0. 50.  0.] 3.0
1546 38650
[ 0.  0.  0. 50.  0.] 3.0
1547 38675
[ 0.  0.  0. 50.  0.] 3.0
1548 38700
[ 0.  0.  0. 50.  0.] 3.0
1549 38725
[ 0.  0.  0. 50.  0.] 3.0
1550 38750
[ 0.  0.  0. 50.  0.] 3.0
1

In [68]:
cube_t = tf.convert_to_tensor(cube)
print('cube_t: ', type(cube_t), cube_t.shape)
cube_d = tf.data.Dataset.from_tensor_slices(cube_t)
print('X: ', type(X))

cube_t:  <class 'tensorflow.python.framework.ops.Tensor'> (1831, 50, 113)
X:  <class 'tensorflow.python.framework.ops.Tensor'>


# CLASSIFICATION #

In [71]:
# parameters
batch_size = 6

In [72]:
# placeholders
X = tf.placeholder(tf.float32, shape=[None, window_size, num_features], name='input')
y = tf.placeholder(tf.float32, shape=[None, 1], name='label')

# layers
conv_1 = tf.layers.conv1d(inputs=X, filters=64, kernel_size=2, activation=tf.nn.relu)
max_pool_1 = tf.layers.max_pooling1d(inputs=conv_1, pool_size=2, strides=2, padding='same')
dropout_1 = tf.layers.dropout(inputs=max_pool_1, rate=0.3)

conv_2 = tf.layers.conv1d(inputs=dropout_1, filters=128, kernel_size=2, activation=tf.nn.relu)
max_pool_2 = tf.layers.max_pooling1d(inputs=conv_2, pool_size=2, strides=2, padding='same')
dropout_2 = tf.layers.dropout(inputs=max_pool_2, rate=0.3)

full_1 = tf.layers.dense(inputs=dropout_2, units=10)

y_pred = tf.layers.dense(inputs=full_1, units=classes)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=lab_cum, logits=y_pred))

# optimizer
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

ValueError: Dimension size must be evenly divisible by 12 but is 1 for 'gradients_2/softmax_cross_entropy_with_logits_2/Reshape_2_grad/Reshape' (op: 'Reshape') with input shapes: [?,12], [1] and with input tensors computed as partial shapes: input[1] = [1].

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((features,labels))
print(type(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


In [16]:
# session

init = tf.global_variables_initializer()
epochs = 100

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(epochs):
        
        iterator = dataset.make_one_shot_iterator()
        batch_x, batch_y = iterator.get_next()
        
        sess.run(train, feed_dict={X:'batch_x', y:'batch_y'})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))

            acc = tf.reduce_mean(tf.cast(matches, tf.float32))

            print(sess.run(acc,feed_dict={X:batch_x, y:batch_y}))
            print('\n')

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("IteratorGetNext:0", shape=(113,), dtype=float64) which was passed to the feed with key Tensor("input:0", shape=(?, 50, 113), dtype=float32).